# Web Content MCP Demo

This notebook demonstrates how to use the Web Content MCP server with AutoGen to create a workflow that:

1. Fetches web content from a URL
2. Rewrites the content in a tech news style
3. Optionally writes the rewritten content to a file

The Web Content MCP server provides a tool for fetching content from web pages, which can then be processed by AutoGen agents.

## Setup

Before running the examples, make sure you have:

1. Installed the required dependencies:
   ```
   pip install -r requirements.txt
   ```

2. Built the TypeScript MCP server:
   ```
   npm install
   npm run build
   ```

3. Created a `.env` file with your OpenAI API key:
   ```
   OPENAI_API_KEY=your_openai_api_key_here
   ```

In [ ]:
# Import common libraries used across all examples
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_core import CancellationToken
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check if OpenAI API key is set
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Warning: OPENAI_API_KEY not set in .env file")
    
# Helper function to truncate content to avoid context length issues
def truncate_content(content, max_chars=10000):
    """Truncate content to avoid context length issues"""
    if len(content) > max_chars:
        return content[:max_chars] + "\n\n[Content truncated due to length...]\n"
    return content

## Example 1: Basic Fetch and Rewrite

This example demonstrates the basic workflow:
1. Set up the MCP fetch server
2. Create two agents: one for fetching content and one for rewriting it
3. Run the workflow to fetch and rewrite content from a URL

The workflow uses a round-robin approach where agents take turns processing the task.

In [ ]:
async def example1():
    # Setup the MCP fetch server parameters
    fetch_mcp_server = StdioServerParams(command="node", args=["./build/index.js"])
    
    # Get the fetch tool from the MCP server
    tools = await mcp_server_tools(fetch_mcp_server)
    
    # Create fetch agent with the MCP fetch tool
    fetch_agent = AssistantAgent(
        name="content_fetcher",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        tools=tools,  # The MCP fetch tool will be included here
        system_message="""You are a web content retrieval assistant. Use the fetch tool to get web content.
        
        IMPORTANT: To avoid context length issues, only fetch a single article or a small section of a page.
        If the content is too large, extract only the main article text or first few paragraphs.
        
        After fetching, summarize the content in 2-3 paragraphs before passing it to the rewriter."""
    )
    
    # Create rewriter Agent
    rewriter_agent = AssistantAgent(
        name="content_rewriter",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        system_message="""You are a content rewriting expert. Rewrite the web content provided to you in a tech news style.
        Tech news style characteristics:
        1. Concise, attention-grabbing headline
        2. Direct opening that states the main point
        3. Objective but engaging content
        4. Use of technical terms with clear explanations
        5. Short paragraphs with emphasized key points
        
        Keep your rewrite concise, focusing on the most important information.
        When you complete the rewrite, reply with TERMINATE."""
    )
    
    # Set up termination condition and team
    termination = TextMentionTermination("TERMINATE")
    team = RoundRobinGroupChat([fetch_agent, rewriter_agent], termination_condition=termination)
    
    # Run the workflow with a specific article URL instead of a category page
    result = await team.run(
        task="Fetch content from https://www.theverge.com/2023/3/14/23639313/google-ai-features-gmail-docs-cloud-smart-chips and rewrite it in a tech news style",
        cancellation_token=CancellationToken()
    )
    
    print("\nFinal rewritten content:\n")
    print(result.messages[-1].content)
    return result

# Run Example 1
# await example1()

## Example 2: With UI Console

This example builds on Example 1 by adding a UI console to display the conversation between agents. This makes it easier to follow the workflow and understand how the agents interact with each other.

In [ ]:
from autogen_agentchat.ui import Console

async def example2():
    # Setup the MCP fetch server parameters
    fetch_mcp_server = StdioServerParams(command="node", args=["./build/index.js"])
    
    # Get the fetch tool from the MCP server
    tools = await mcp_server_tools(fetch_mcp_server)
    
    # Create fetch agent with the MCP fetch tool
    fetch_agent = AssistantAgent(
        name="content_fetcher",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),  # Using mini model to reduce token usage
        tools=tools,
        system_message="""You are a web content retrieval assistant. Use the fetch tool to get web content.
        
        IMPORTANT: To avoid context length issues, only fetch a single article or a small section of a page.
        If the content is too large, extract only the main article text or first few paragraphs.
        
        After fetching, summarize the content in 2-3 paragraphs before passing it to the rewriter."""
    )
    
    # Create rewriter Agent
    rewriter_agent = AssistantAgent(
        name="content_rewriter",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),  # Using mini model to reduce token usage
        system_message="""You are a content rewriting expert. Rewrite the web content provided to you in a tech news style.
        Tech news style characteristics:
        1. Concise, attention-grabbing headline
        2. Direct opening that states the main point
        3. Objective but engaging content
        4. Use of technical terms with clear explanations
        5. Short paragraphs with emphasized key points
        
        Keep your rewrite concise, focusing on the most important information.
        When you complete the rewrite, reply with TERMINATE."""
    )
    
    # Set up termination condition and team
    termination = TextMentionTermination("TERMINATE")
    team = RoundRobinGroupChat([fetch_agent, rewriter_agent], termination_condition=termination)
    
    # Use a specific article URL instead of a category page
    task = "Fetch content from https://techcrunch.com/2023/03/14/microsoft-brings-openais-dall-e-image-creator-to-the-new-bing/ and rewrite it in a tech news style"
    
    # Display the conversation process
    stream = team.run_stream(task=task, cancellation_token=CancellationToken())
    await Console(stream)
    
    # Get the final result (using a separate run call)
    result = await team.run(task=task, cancellation_token=CancellationToken())
    
    print("\nFinal rewritten content:\n")
    print(result.messages[-1].content)
    
    return result

# Run Example 2
# await example2()

## Example 3: With File Writing

This example extends the workflow by adding a third agent that writes the rewritten content to a file. It demonstrates how to use multiple MCP servers together (the custom fetch server and the standard filesystem server).

In [ ]:
from datetime import datetime

async def example3():
    # Setup the MCP fetch server parameters
    # This server is used to fetch web content
    fetch_mcp_server = StdioServerParams(command="node", args=["./build/index.js"])

    # Setup the MCP filesystem server parameters
    # This server is used to write to local files
    write_mcp_server = StdioServerParams(command="npx", args=["-y", "@modelcontextprotocol/server-filesystem", "."])
    
    # Get the fetch tool from the MCP server
    tools_fetch = await mcp_server_tools(fetch_mcp_server)

    # Get the filesystem tool from the MCP server
    tools_write = await mcp_server_tools(write_mcp_server)
    
    # Create content fetcher agent
    # This agent is responsible for fetching web content
    fetch_agent = AssistantAgent(
        name="content_fetcher",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        tools=tools_fetch,
        system_message="""You are a web content retrieval assistant. Use the fetch tool to get web content.
        
        IMPORTANT: To avoid context length issues, only fetch a single article or a small section of a page.
        If the content is too large, extract only the main article text or first few paragraphs.
        
        After fetching, summarize the content in 2-3 paragraphs before passing it to the rewriter."""
    )
    
    # Create content rewriter agent
    # This agent is responsible for rewriting web content in tech news style
    # Note: No longer adding TERMINATE at completion, but passing content to the next agent    
    rewriter_agent = AssistantAgent(
        name="content_rewriter",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        system_message="""You are a content rewriting expert. Rewrite the web content provided to you in a tech news style.
        Tech news style characteristics:
        1. Concise, attention-grabbing headline
        2. Direct opening that states the main point
        3. Objective but engaging content
        4. Use of technical terms with clear explanations
        5. Short paragraphs with emphasized key points
        
        Keep your rewrite concise, focusing on the most important information.
        When you complete the rewrite, pass the content to the content_writer agent to save it to a file."""
    )
    
    # Get current date and format as YYYY-MM-DD
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Create file writer agent
    # This agent is responsible for writing the rewritten content to a local file
    # Note: This agent will add TERMINATE after completing its task to end the conversation
    write_agent = AssistantAgent(
        name="content_writer",
        model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
        tools=tools_write,
        system_message=f"""You are a file assistant. Use the filesystem tool to write the content provided by content_rewriter to a text file named with today's date ({current_date}.txt).
        When you successfully write the file, reply with "TERMINATE" to end the conversation."""
    )
    
    # Set up termination condition and team
    # When any agent replies with TERMINATE, the conversation will end
    termination = TextMentionTermination("TERMINATE")
    team = RoundRobinGroupChat([fetch_agent, rewriter_agent, write_agent], termination_condition=termination)
    
    # Use a specific article URL instead of a category page
    task = "Fetch content from https://www.wired.com/story/chatgpt-writing-test-education/, rewrite it in a tech news style, and save the rewritten article to a local text file"
    
    # Execute the task once using the run method
    result = await team.run(task=task, cancellation_token=CancellationToken())
    
    # Print the final result
    print(f"\nFile has been written to {current_date}.txt")
    
    return result

# Run Example 3
# await example3()

## Running the Examples

To run any of the examples, uncomment the corresponding await line at the end of each example function. For instance, to run Example 1, uncomment the line `# await example1()`.

You can run the examples one at a time to see how each works. Below is a simple way to run Example 1:

In [ ]:
# Run Example 1 (Basic fetch and rewrite)
await example1()

## Conclusion

This notebook has demonstrated three examples of using the Web Content MCP server with AutoGen:

1. **Basic Example**: Fetching and rewriting web content
2. **UI Console Example**: Adding a UI to visualize the conversation between agents
3. **File Writing Example**: Extending the workflow to save the rewritten content to a file

These examples showcase how MCP servers can be used to extend the capabilities of AutoGen agents, allowing them to interact with external systems like web content and the filesystem.

### Key Concepts

- **MCP Servers**: Provide tools that agents can use to interact with external systems
- **AutoGen Agents**: Specialized AI assistants that can use tools and communicate with each other
- **Round-Robin Teams**: A way to organize agents to take turns processing a task
- **Termination Conditions**: Rules that determine when a conversation should end

### Next Steps

You can extend these examples by:
- Adding more agents to the workflow
- Implementing different conversation patterns
- Creating custom MCP servers for other external systems
- Building a web UI for the workflow